In [1]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 672 kB/s eta 0:00:01


In [2]:
!pip install h5py

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [1]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
import pandas as pd
import numpy as np
import tqdm
import time
from gensim.models import Word2Vec

np.random.seed(2020)


print('loading trian data ... \n\n')
train_click = pd.read_csv('../../train_preliminary/click_log.csv').sort_values(by = ['user_id','time'])
train_ad = pd.read_csv('../../train_preliminary/ad.csv')
train_click = train_click.merge(train_ad,on = 'creative_id',how = 'left')
train_click = train_click[['user_id','time','ad_id']].sort_values(by = ['user_id','time'])
train_click = train_click.set_index('user_id')
print('loading trian data  over  ... \n\n')


print('loading test data ... \n\n')
test_click = pd.read_csv('../../test/click_log.csv').sort_values(by = ['user_id','time'])
test_ad = pd.read_csv('../../test/ad.csv')
test_click = test_click.merge(test_ad,on = 'creative_id',how = 'left')
test_click = test_click[['user_id','time','ad_id']].sort_values(by = ['user_id','time'])
test_click = test_click.set_index('user_id')
print('loading test data  over  ... \n\n')


def get_str_data():
    text_data = []
    for i in tqdm.tqdm(range(1, 900001)):
#     for i in tqdm.tqdm(range(1, 9001)):
        train_user_ad = train_click.loc[i, 'ad_id'].values
        text = [str(x) for x in train_user_ad]
        text_data.append(text)
    for i in tqdm.tqdm(range(3000001, 4000001)):
#     for i in tqdm.tqdm(range(3000001, 3001001)):
        test_user_ad = test_click.loc[i, 'ad_id'].values
        text = [str(x) for x in test_user_ad]
        text_data.append(text)
    return text_data

print('get list data ...\n\n')
data = get_str_data()

loading trian data ... 


loading trian data  over  ... 


loading test data ... 




  0%|          | 0/900000 [00:00<?, ?it/s]

loading test data  over  ... 


get list data ...




100%|██████████| 1000000/1000000 [08:35<00:00, 1939.75it/s]


In [3]:

# del train_click
# del test_click
# del train_ad
# del test_ad


model = Word2Vec(data, size=32, window=200, min_count=5, workers=8, sg=1)

print('saving model ... \n\n')
model.save('word2vec_ad_id_pre_embed_size_32_w200_count_0')
print('model saved ... \n\n')

saving model ... 


model saved ... 




In [4]:

# model = Word2Vec.load("word2vec_ad_id_pre_embed_size_32_w50_count_0")
#1,导入w2v模型
# w2v_model = Word2Vec.load("../../get_w2v_feat/word2vec.model")
## 2 构造包含所有词语的 list，以及初始化 “词语-序号”字典 和 “词向量”矩阵
vocab_list = [word for word, Vocab in model.wv.vocab.items()]# 存储 所有的 词语
word_index = {" ": 0}# 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_vector = {} # 初始化`[word : vector]`字典
# 初始化存储所有向量的大矩阵，留意其中多一位（首行），词向量全为 0，用于 padding补零。
# 行数 为 所有单词数+1 比如 10000+1 ； 列数为 词向量“维度”比如100。
embeddings_matrix = np.zeros((len(vocab_list) + 1, model.vector_size))
## 3 填充 上述 的字典 和 大矩阵
for i in tqdm.tqdm(range(len(vocab_list))):
    # print(i)
    word = vocab_list[i]  # 每个词语
    word_index[word] = i + 1 # 词语：序号
    word_vector[word] = model.wv[word] # 词语：词向量
    embeddings_matrix[i + 1] = model.wv[word]  # 词向量矩阵

import h5py
with h5py.File('embeddings_matrix_ad_size_32_w200_count_0.h5', 'w') as hf:
    hf.create_dataset("embeddings_matrix",  data=embeddings_matrix)


100%|██████████| 1009977/1009977 [00:07<00:00, 140533.72it/s]


In [5]:
# 序号化 文本，tokenizer句子，并返回每个句子所对应的词语索引
from keras.preprocessing import sequence

texts = data
data_pad = []
for sentence in tqdm.tqdm(texts):
    new_txt = []
    for word in sentence:
        try:
            new_txt.append(word_index[word])  # 把句子中的 词语转化为index
        except:
            new_txt.append(0)

    data_pad.append(new_txt)
    
MAX_SEQUENCE_LENGTH = 200
texts_pad = sequence.pad_sequences(data_pad, maxlen = MAX_SEQUENCE_LENGTH)  # 使用kears的内置函数padding对齐句子,好处是输出numpy数组，不用自己转化了


train_data = texts_pad[:900000,:]
test_data = texts_pad[900000:,:]
with h5py.File('word_train_ad_w2v_w200.h5', 'w') as hf:
    hf.create_dataset("word_data",  data=train_data)
with h5py.File('word_test_ad_w2v_w200.h5', 'w') as hf:
    hf.create_dataset("word_data",  data=test_data)


Using TensorFlow backend.
100%|██████████| 1900000/1900000 [00:48<00:00, 38824.44it/s]
